# Neural Network Training and Backpropagation

This notebook modifies a 2-layered neural network to a 3-layered neural network with scaled sigmoid activation functions. It includes forward propagation, cost calculation, and backpropagation with and without regularization.

## Overview
The key steps involve updating functions for scaled sigmoid, modifying forward propagation and cost calculation for additional layers, implementing backpropagation, and training the model.

## Procedure
- **Scaled Sigmoid Function**: Modify `sigmoid()` to return scaled sigmoid.
- **Forward Propagation**: Update `forward_propagate()` for two hidden layers.
- **Cost Calculation**: Update `cost()` for predictions from the 3-layered neural network with and without regularization.
- **Sigmoid Gradient**: Modify `sigmoid_gradient()` to return gradient of scaled sigmoid function.
- **Backpropagation**: Update `backprop()` to compute gradients and implement two versions, with and without regularization.
- **Model Training**: Train the 3-layered neural network by minimizing the objective function.
- **Model Accuracy**: Make forward predictions and compute accuracy.
- **Comparison**: Compare model accuracy with the 2-layered neural network.

In [3]:
print('x')
x=4
x

print(4)


x
4


In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

data = loadmat('../data/ex3data1.mat')
data


{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

Since we're going to need these later (and will use them often), let's create some useful variables up-front.

In [65]:
X = data["X"]
y = data["y"]

X.shape, y.shape


((5000, 400), (5000, 1))

We're also going to need to one-hot encode our y labels.  One-hot encoding turns a class label n (out of k classes) into a vector of length k where index n is "hot" (1) while the rest are zero.  Scikit-learn has a built in utility we can use for this.

In [66]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape


(5000, 10)

In [67]:
y[0], y_onehot[0, :]


(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

### OG Description
The neural network we're going to build for this exercise has an input layer matching the size of our instance data (400 + the bias unit), a hidden layer with 25 units (26 with the bias unit), and an output layer with 10 units corresponding to our one-hot encoding for the class labels.  For additional details and an image of the network architecture, please refer to the PDF in the "exercises" folder.

The first piece we need to implement is a cost function to evaluate the loss for a given set of network parameters.  The source mathematical function is in the exercise text (and looks pretty intimidating).  Here are the functions required to compute the cost.


### Current Description

Next, in the template provided, you will make a copy of this notebook and modify it to train a 3-layered neural network with two hidden layers using the same dataset. The number of hidden units in the first and second hidden layers is 20 and 20. The activation function you will use in hidden layers is scaled sigmoid, given by:

$$ \hat{\sigma}(z) = \frac{1}{1 + e^{-2z}} $$

You will need to make changes to the following functions in the original notebook.

1. `sigmoid()` to return scaled sigmoid. (1)
2. `forward_propagate()` to account for 2 hidden layers (the original has one hidden layer). (3)
3. `cost()` to calculate predictions from the 3-layered neural network and hence the cost. You need to make changes to both versions of the `cost()` function, with and without regularization, as in the original notebook. (4)
4. `sigmoid_gradient()` to return gradient of scaled sigmoid function. (2)
5. `backprop()` to compute the gradients. Your function should return both the cost and the gradient vector, as in the original notebook. Also, you will need to implement two versions of these functions, with and without regularization, as in the original notebook. (8)

Then you will

6. Train your 3-layered neural network by minimizing the objective function, as in the original notebook, keeping the hyperparameters (learning rate, method, `jac`, options) unchanged. (3)
7. Make forward predictions from your trained model and compute the accuracy. (2)
8. How does your model accuracy compare with the accuracy of the 2-layered neural network in the original notebook? (2)


### 2.1 Custom Sigmoid


In [68]:
# Math: \sigma(z) = \frac{1}{1 + e^{-2z}}

def new_sigmoid(z):
    return 1 / (1 + np.exp(-2 * z))


### 2.2 Custom Propagate

In [69]:
def new_forward_propagate(X, theta1, theta2, theta3):
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)

    z2 = a1 * theta1.T
    a2 = np.insert(new_sigmoid(z2), 0, values=np.ones(m), axis=1)

    z3 = a2 * theta2.T
    a3 = np.insert(new_sigmoid(z3), 0, values=np.ones(m), axis=1)

    z4 = a3 * theta3.T
    h = new_sigmoid(z4)

    return a1, z2, a2, z3, a3, z4, h


### 2.3 Custom Cost No Reg 

In [81]:
def new_cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # Reshape the parameter array into parameter matrices for each layer


    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1)], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))


    # Run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = new_forward_propagate(X, theta1, theta2, theta3)

    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)

    J = J / m
    return J


We've used the sigmoid function before so that's not new.  The forward-propagate function computes the hypothesis for each training instance given the current parameters.  It's output shape should match the same of our one-hot encoding for y.  We can test this real quick to convince ourselves that it's working as expected (the intermediate steps are also returned as these will be useful later).

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

a1.shape, z2.shape, a2.shape, z3.shape, h.shape


The cost function, after computing the hypothesis matrix h, applies the cost equation to compute the total error between y and h.

cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)


#### Test new functions


In [ ]:
# test for the new to train a 3-layered neural network with two hidden layers using the same dataset. The number of hidden units in the first and second hidden layers is 20 and 20. The activation function you will use in hidden layers is scaled sigmoid

# initial setup
input_size = 400
hidden_size = 20
num_labels = 10
learning_rate = .9

print(f"==>> learning_rate: {learning_rate}")

# randomly initialize a parameter array of the size of the full network's parameters
params = (
    np.random.random(
        size=hidden_size * (input_size + 1)
        + hidden_size * (hidden_size + 1)
        + num_labels * (hidden_size + 1)
    )
    - 0.5
) * 0.25


# params = (
#     np.random.random(
#         size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)
#     )
#     - 0.5
# ) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# initialize theta 1, 2 and 3
# theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
# theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1)], (hidden_size, (hidden_size + 1))))
# theta3 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))
# theta3 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1)], (hidden_size, (hidden_size + 1))))
theta3 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape, theta3.shape

np.matrix(np.reshape(params[hidden_size * (input_size + 1) + hidden_size * (3 + 1):], (num_labels, (hidden_size + 1))))

print("m shape:", m)
print("X shape:", X.shape)
print("y shape:", y.shape)
print("theta1 shape:", theta1.shape)
print("theta2 shape:", theta2.shape)

print("params shape:", params.shape)




In [83]:
a1, z2, a2, z3, a3, z4, h = new_forward_propagate(X, theta1, theta2, theta3)


new_cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate )


7.323056299376311

Our next step is to add regularization to the cost function.  If you're following along in the exercise text and thought the last equation looked ugly, this one looks REALLY ugly.  It's actually not as complicated as it looks though - in fact, the regularization term is simply an addition to the cost we already computed.  Here's the revised cost function.

### 2.3 Custom Cost with back propagation


In [84]:
def custom_cost_reg(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # Reshape the parameter array into parameter matrices for each layer
    # Assuming the second hidden layer also has the same size as the first hidden layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1)], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))


    # Run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = new_forward_propagate(X, theta1, theta2, theta3)

    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)


    # with reg
    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))

    return J



Next up is the backpropagation algorithm.  Backpropagation computes the parameter updates that will reduce the error of the network on the training data.  The first thing we need is a function that computes the gradient of the sigmoid function we created earlier.

### 2.4 Custom sigmoid_gradient


In [85]:
def new_sigmoid_gradient(z):
    return np.multiply(new_sigmoid(z), (1 - new_sigmoid(z)))



Now we're ready to implement backpropagation to compute the gradients.  Since the computations required for backpropagation are a superset of those required in the cost function, we're actually going to extend the cost function to also perform backpropagation and return both the cost and the gradients.

### 2.5 Custom Back Propagation


In [86]:

def new_backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # Reshape the parameter array into parameter matrices for each layer
    # Assuming the second hidden layer also has the same size as the first hidden layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1)], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))

    # Run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = new_forward_propagate(X, theta1, theta2, theta3)

    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)

    # with reg
    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))

    # perform backpropagation
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (25, 26)
    delta3 = np.zeros(theta3.shape)  # (10, 26)

    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        z3t = z3[t,:]  # (1, 25)
        a3t = a3[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)

        d4t = ht - yt  # (1, 10)

        # Errors for the second hidden layer
        z3t = np.insert(z3t, 0, values=np.ones(1))  # add bias unit
        d3t = np.multiply((theta3.T * d4t.T).T, new_sigmoid_gradient(z3t))

        # Errors for the first hidden layer
        z2t = np.insert(z2t, 0, values=np.ones(1))  # add bias unit
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, new_sigmoid_gradient(z2t))

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + (d3t[:,1:]).T * a2t
        delta3 = delta3 + d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))
    return J, grad


The hardest part of the backprop computation (other than understanding WHY we're doing all these calculations) is getting the matrix dimensions right.  By the way, if you find it confusing when to use A * B vs. np.multiply(A, B), you're not alone.  Basically the former is a matrix multiplication and the latter is an element-wise multiplication (unless A or B is a scalar value, in which case it doesn't matter).  I wish there was a more concise syntax for this (maybe there is and I'm just not aware of it).

Anyway, let's test it out to make sure the function returns what we're expecting it to.

In [87]:
J, grad = new_backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape


(7.327116265985744, (8650,))

We still have one more modification to make to the backprop function - adding regularization to the gradient calculations.  The final regularized version is below.

In [91]:
def new_backprop_reg(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # Reshape the parameter array into parameter matrices for each layer
    # Assuming the second hidden layer also has the same size as the first hidden layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1)], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))

    # Run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = new_forward_propagate(X, theta1, theta2, theta3)

    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)

    # with reg
    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)) + np.sum(np.power(theta3[:,1:], 2)))

    # perform backpropagation
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (25, 26)
    delta3 = np.zeros(theta3.shape)  # (10, 26)

    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        z3t = z3[t,:]  # (1, 25)
        a3t = a3[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)

        d4t = ht - yt  # (1, 10)

        # Errors for the second hidden layer
        z3t = np.insert(z3t, 0, values=np.ones(1))  # add bias unit
        d3t = np.multiply((theta3.T * d4t.T).T, new_sigmoid_gradient(z3t))

        # Errors for the first hidden layer
        z2t = np.insert(z2t, 0, values=np.ones(1))  # add bias unit
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, new_sigmoid_gradient(z2t))

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + (d3t[:,1:]).T * a2t
        delta3 = delta3 + d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    delta3[:,1:] = delta3[:,1:] + (theta3[:,1:] * learning_rate) / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))
    return J, grad


In [92]:
J, grad = new_backprop_reg(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape


(7.161517852858925, (8650,))

## 2.6 


We're finally ready to train our network and use it to make predictions.  This is roughly similar to the previous exercise with multi-class logistic regression.

In [93]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=new_backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),
                method='TNC', jac=True, options={'maxfun': 250})
fmin

fmin_reg = minimize(fun=new_backprop_reg, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),
                method='TNC', jac=True, options={'maxfun': 250})
fmin_reg


 message: Max. number of function evaluations reached
 success: False
  status: 3
     fun: 0.18141891680252428
       x: [ 7.168e-01  1.187e-02 ...  1.804e+00  1.016e+00]
     nit: 26
     jac: [ 4.088e-04  2.136e-06 ... -3.779e-04 -6.056e-04]
    nfev: 251

We put a bound on the number of iterations since the objective function is not likely to completely converge.  Our total cost has dropped below 0.5 though so that's a good indicator that the algorithm is working.  Let's use the parameters it found and forward-propagate them through the network to get some predictions.

## 2.7


In [94]:
def calc_min(fmin, X=X, y=y, input_size=input_size, hidden_size=hidden_size, num_labels=num_labels):
    X = np.matrix(X)

    theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1)], (hidden_size, (hidden_size + 1))))
    theta3 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))

    a1, z2, a2, z3, a3, z4, h = new_forward_propagate(X, theta1, theta2, theta3)

    y_pred = np.array(np.argmax(h, axis=1) + 1)

    correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
    accuracy = (sum(map(int, correct)) / float(len(correct)))

    # print('accuracy = {0}%'.format(accuracy * 100))

    return accuracy * 100

print(f'minimize without regularization: {calc_min(fmin):.2f}%')
print(f'minimize with regularization: {calc_min(fmin_reg):.2f}%')


minimize without regularization: 99.12%
minimize with regularization: 99.14%


### Accuracy Comparison between 2 and 3 hidden layered Neural Network


The 3-Layered Network produced the folowing accuracies:
* minimize without regularization: 99.68%
* minimize with regularization: 99.72%
  
The 2-Layered Network produced the following accuracies:
* minimize with regularization: 99.22%
  
The 3-Layered Network is more accurate than the 2-Layered Network. However, the difference is not that significant. It is only 0.46% more accurate. 

In [95]:
X = np.matrix(X)

theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1)], (hidden_size, (hidden_size + 1))))
theta3 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1) + hidden_size * (hidden_size + 1):], (num_labels, (hidden_size + 1))))
a1, z2, a2, z3, a3, z4, h = new_forward_propagate(X, theta1, theta2, theta3)

y_pred = np.array(np.argmax(h, axis=1) + 1)


correct = [0 if a == b else 1 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))



accuracy = 0.88%


Finally we can compute the accuracy to see how well our trained network is doing.

## 2.8


In [96]:
correct = [0 if a == b else 1 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))


accuracy = 0.88%


And we're done!  We've successfully implemented a rudimentary feed-forward neural network with backpropagation and used it to classify images of handwritten digits.  In the next exercise we'll look at another power supervised learning algorithm, support vector machines.